<a id="id1000"></a> 

<a href ="#contents"> 
<p style="text-align: right;">return to contents</p> 
</a> 
 
# Section 1: Tools:

These are by no means finished or completely robust and some sections are still drafts

These tools are aimed to help understand the structure of the code and the context of functions. With the preliminary aim of aiding cython wrapping and the end goal of some level of wrapping automation. 

This may be useful for the following reasons

* Work on SEVN is still active so it may be cumbersome to wrap manually every major git update
* Errors are prone to happen if changes to every small part is applied repeatedly and manually through the code 
* To learn better understand the relationships between functions of the code. 

<a id="contents"></a> 
# Contents 
  * [ Section 1: Tools:](#id1000)  
    * [ 1.1:  Code Searching Tools:  Basic Tools](#id1100)  
      * [ 1.1.1:  Simple Functions (just return file list) ](#id1110)  
      * [ 1.1.2:  Dictionary of attributes for context](#id1120)  
    * [ 1.2:  Function Finding](#id1200)  

In [1]:
import os
import re
SEVN = '/home/jake/Downloads/sevndevel-SEVN'

<a id="id1100"></a> 

<a href ="#contents"> 
<p style="text-align: right;">return to contents</p> 
</a> 
 
## 1.1:  Code Searching Tools:  Basic Tools

In [2]:
def header_dependencies(header_path):
    f = open(header_path , "r")
    sub_header_files = []
    header_syntax = '<>'
    for line in f:
        hashtag_list = re.findall("#(\w+)", line)#not elegent but "#include does not work"
        if len(hashtag_list)>0 and 'include' and '.h' in line:#we now find either c++ header syntaxs <> or ""
            if header_syntax[0] not in line  and header_syntax[1] not in line: header_syntax='""'
            if header_syntax[0] in line  and header_syntax[1] in line:
                result = re.search(header_syntax[0]+'(.*)'+header_syntax[1], line)#remove formatting
                sub_header_files.append(result.group(1)) 
    f.close()
    return sub_header_files

def find_all_headers(dir_path):#h file contains shared declarations
    all_headers = []
    for root, dirs, files in os.walk(SEVN):
        for f in files:
            if f.endswith('.h'): 
                all_headers.append(os.path.join(root,f))
    return all_headers

def check_comment(line, block_open=False):
    ## in this function we just return the start and endpoints of the valid code and a
    ## bool stating if there is an open block to consider
    ## could make more robust ie against weird exaples ie 'abcd/*s ncj */ fh /*'
    line_comment = line.find('//') #-1 if not found
    block_comment = line.find('/*')
    block_end = line.find('*/')
    code_pos=(0, len(line))
    comment=False
    
    if line_comment ==-1 and block_comment == -1: #no comment
        code_pos=(0, len(line)) #no position or change to block opening
        comment = True 
    
    if line_comment >-1 and block_comment == -1: #line comment
        code_pos= (0,line_comment)
    
    if line_comment ==-1 and block_comment > -1: #block comment found 
        #print('block found')
        if not block_open:
            block_open=True # could check to see order if open close as 
        code_pos=(0,block_comment-1)
    
    if block_open:# check if block already open
        if block_end > -1: #check block close
            #print('block closed')
            block_open = False
            code_pos = (block_end,len(line))
            
    if block_comment < block_end:
        code_pos = (0,block_comment)
        
    return code_pos, comment, block_open

def find_all_cpp(dir_path):
    all_cpp = []
    for root, dirs, files in os.walk(SEVN):
        for f in files:
            if f.endswith('.cpp'): 
                all_cpp.append(os.path.join(root,f))
    return all_cpp

def next_word(line, word): #get the next word in string eg after namespace or class 
    words = line.split()
    next_word = words[words.index(word) + 1]
    next_word = next_word.replace('{','')
    return next_word.replace(';','')

def last_word(line, word):
    words = line.split()
    next_word = words[-1]
    next_word = next_word.replace('{','')
    return next_word.replace(';','')

# keep track of scope eg in namespaces 
def bracket_counter(line, l=0, r=0, closed=False, brackets='{}'):
    for ch in line:
        if ch == brackets[0]:l+=1
        if ch == brackets[1]:r+=1
        if r==l and l!=0 and r>0 : closed= True
    return l, r, closed

def finder(file_path, what='namespace', show_line =False): 
    #what can be namespace class
    if type(file_path) is not list: file_path = [file_path] 
    properties ={}
    for path in file_path:
        f = open(path , "r")

        l,r, closed =0,0,True 
        line_nums = None#ie no span just in same format

        i=1
        block_open=False
        for line in f:
            pos, comment, block_open=check_comment(line, block_open)
            line=line[pos[0]:pos[1]]
            if not block_open:
                if what in line:
                    if 'template' not in line:
                        if what=='typedef': out=last_word(line, what)
                        else: out=next_word(line, what)
                        if '{' in line:#scope handling
                            l,r, closed = bracket_counter(line)
                            start_line = i # bracket found here
                        else:# no scope from brackets
                            line_nums = (i,i)
                            #if show_line: out= 'l='+str(i)+': '+out
                            properties[out]= {'file': path ,'lines': line_nums}                    
                else:
                    if not closed:
                        if start_line!=i: l,r, closed = bracket_counter(line, l, r, closed)
                        if closed:# 
                            line_nums = (start_line, i)
                            properties[out]= {'file': path ,'lines': line_nums}
                i+=1
        f.close()      
    return properties

def class_info(dir_list):
    class_properties ={}
    if type(dir_list) is not list: dir_list = [dir_list]#can accept one or more files
    for path in dir_list:
        namespaces = finder(path, 'namespace')
        cls  = finder(path, 'class')
        #Extract key parts
        ns_list  = [[key, namespaces[key]['lines']] for key in namespaces]
        cls_list = [[key, cls[key]['lines']] for key in cls]

        #check namespace 
        for ns in ns_list:#usually only one but wrote genral 
            ns_min, ns_max = ns[1]
            for c in cls_list:
                c_min, c_max = c[1]
                if ns_min<c_min  and c_max < ns_max or ns_min ==ns_max: 
                    class_properties[c[0]]={'namespace': ns[0], 'file': path, 'lines': c[1]}
                else:
                    class_properties[c[0]]={'namespace': None, 'file': path, 'lines': c[1]}
    return class_properties

def save_list(input_list, file_name):
    with open(file_name, 'w') as fp:
        fp.write('\n'.join(input_list))

def save_dict(input_dict, file_name):
    import json
    #input_dict=dict(input_dict)
    with open(file_name, 'w') as convert_file:
        convert_file.write(json.dumps(input_dict, indent=2))
    
def load_list(file_path):
    f = open(file_path , "r")
    file_list=[line for line in f]
    f.close()
    return file_list

def load_dict(file_path):
    with open(file_path) as json_file:
        data = json.load(json_file)

<a id="id1110"></a> 

<a href ="#contents"> 
<p style="text-align: right;">return to contents</p> 
</a> 
 
### 1.1.1:  Simple Functions (just return file list) 

In [3]:
from pprint import pprint

all_h = find_all_headers(SEVN)
all_cpp = find_all_cpp(SEVN)
save_list(all_h, 'all_headers.txt')
save_list(all_cpp, 'all_cpp.txt')

print('header files')
pprint(all_h)

print('\n cpp files')
all_cpp

header files
['/home/jake/Downloads/sevndevel-SEVN/include/general/params.h',
 '/home/jake/Downloads/sevndevel-SEVN/include/general/IO.h',
 '/home/jake/Downloads/sevndevel-SEVN/include/general/starparameter.h',
 '/home/jake/Downloads/sevndevel-SEVN/include/general/types.h',
 '/home/jake/Downloads/sevndevel-SEVN/include/general/lookup_and_phases.h',
 '/home/jake/Downloads/sevndevel-SEVN/include/general/static_main.h',
 '/home/jake/Downloads/sevndevel-SEVN/include/general/utils/evolve.h',
 '/home/jake/Downloads/sevndevel-SEVN/include/general/utils/sevnlog.h',
 '/home/jake/Downloads/sevndevel-SEVN/include/general/utils/errhand.h',
 '/home/jake/Downloads/sevndevel-SEVN/include/general/utils/utilities.h',
 '/home/jake/Downloads/sevndevel-SEVN/include/h5out/H5out.h',
 '/home/jake/Downloads/sevndevel-SEVN/include/binstar/binstar.h',
 '/home/jake/Downloads/sevndevel-SEVN/include/binstar/Orbit.h',
 '/home/jake/Downloads/sevndevel-SEVN/include/binstar/qcrit.h',
 '/home/jake/Downloads/sevndevel-S

['/home/jake/Downloads/sevndevel-SEVN/build/CMakeFiles/FindOpenMP/OpenMPCheckVersion.cpp',
 '/home/jake/Downloads/sevndevel-SEVN/build/CMakeFiles/FindOpenMP/OpenMPTryFlag.cpp',
 '/home/jake/Downloads/sevndevel-SEVN/build/CMakeFiles/3.22.1/CompilerIdCXX/CMakeCXXCompilerId.cpp',
 '/home/jake/Downloads/sevndevel-SEVN/main/sevnB.cpp',
 '/home/jake/Downloads/sevndevel-SEVN/main/sevnB_oldversion.cpp',
 '/home/jake/Downloads/sevndevel-SEVN/main/sevn.cpp',
 '/home/jake/Downloads/sevndevel-SEVN/main/sevn_oldversion.cpp',
 '/home/jake/Downloads/sevndevel-SEVN/sevnlib_examples/binary_evol/binary_evol.cpp',
 '/home/jake/Downloads/sevndevel-SEVN/src/general/IO.cpp',
 '/home/jake/Downloads/sevndevel-SEVN/src/general/lookup_and_phases.cpp',
 '/home/jake/Downloads/sevndevel-SEVN/src/general/params.cpp',
 '/home/jake/Downloads/sevndevel-SEVN/src/general/starparameter.cpp',
 '/home/jake/Downloads/sevndevel-SEVN/src/general/utils/utilities.cpp',
 '/home/jake/Downloads/sevndevel-SEVN/src/general/utils/evo

<a id="id1120"></a> 

<a href ="#contents"> 
<p style="text-align: right;">return to contents</p> 
</a> 
 
### 1.1.2:  Dictionary of attributes for context

In [4]:
all_namespaces=finder(all_h, 'namespace')
all_classes=finder(all_h, 'class')

print('All Namespaces') 
pprint(all_namespaces)
print('All Classes')
pprint(all_classes) 

save_dict(all_namespaces, 'all_namespaces.json')
save_dict(all_classes, 'all_classes.json')

All Namespaces
{'Lookup': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/star/property.h',
            'lines': (19, 19)},
 'evolve_utility': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/general/utils/evolve.h',
                    'lines': (28, 919)},
 'sevnstd': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/h5out/H5out.h',
             'lines': (19, 58)},
 'starparameter': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/general/starparameter.h',
                   'lines': (13, 25)},
 'std': {'file': '/home/jake/Downloads/sevndevel-SEVN/hdf5_interpreter/python_cxx/plot_matplotlib.h',
         'lines': (9, 9)},
 'utilities': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/general/utils/utilities.h',
               'lines': (32, 765)}}
All Classes
{'AngMom': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/binstar/BinaryProperty.h',
            'lines': (256, 288)},
 'AngMomSpin': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/star/proper

We can combine this infomation to figure out contextual inforamtion like the namespace of a class

In [5]:
c_info=class_info(all_h)
print('Classes Info')
pprint(c_info)
save_dict(c_info, 'class_info.json')

Classes Info
{'AngMomSpin': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/star/property.h',
                'lines': (1367, 1394),
                'namespace': 'Lookup'},
 'Binstar': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/star/property.h',
             'lines': (32, 32),
             'namespace': 'Lookup'},
 'Bmag': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/star/property.h',
          'lines': (1190, 1213),
          'namespace': 'Lookup'},
 'ConvectiveTable': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/star/property.h',
                     'lines': (1091, 1096),
                     'namespace': 'Lookup'},
 'CoreRadius': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/star/property.h',
                'lines': (860, 874),
                'namespace': 'Lookup'},
 'Csup': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/star/property.h',
          'lines': (1015, 1038),
          'namespace': 'Lookup'},
 'Depthconv': {'file': '/ho

In [6]:
all_typedefs =finder(all_h, 'typedef')
type_def_list = [key for key in all_typedefs]
save_dict(all_typedefs, 'type_defs.json')
print('Type defs')
all_typedefs

Type defs


{'num_entry': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/general/params.h',
  'lines': (349, 349)},
 'str_entry': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/general/params.h',
  'lines': (350, 350)},
 'bool_entry': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/general/params.h',
  'lines': (351, 351)},
 'map_num': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/general/params.h',
  'lines': (352, 352)},
 'map_str': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/general/params.h',
  'lines': (353, 353)},
 'map_bool': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/general/params.h',
  'lines': (354, 354)},
 'OUTPUTMAP': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/general/lookup_and_phases.h',
  'lines': (190, 190)},
 '_INPAIR': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/general/lookup_and_phases.h',
  'lines': (201, 201)},
 'INPUTMAPBIN_NPARAM': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/general/lookup_and_

In [7]:
all_namespaces

{'Lookup': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/star/property.h',
  'lines': (19, 19)},
 'starparameter': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/general/starparameter.h',
  'lines': (13, 25)},
 'evolve_utility': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/general/utils/evolve.h',
  'lines': (28, 919)},
 'sevnstd': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/h5out/H5out.h',
  'lines': (19, 58)},
 'utilities': {'file': '/home/jake/Downloads/sevndevel-SEVN/include/general/utils/utilities.h',
  'lines': (32, 765)},
 'std': {'file': '/home/jake/Downloads/sevndevel-SEVN/hdf5_interpreter/python_cxx/plot_matplotlib.h',
  'lines': (9, 9)}}

<a id="id1200"></a> 

<a href ="#contents"> 
<p style="text-align: right;">return to contents</p> 
</a> 
 
## 1.2:  Function Finding

Declaration and Definition

A C++ function definition consists of two parts:

    Declaration: the return type, the name of the function, and parameters (if any)
    Definition: the body of the function (code to be executed)

Example
```cpp
int main() {
  myFunction();
  return 0;
}
```
Therefore they can be found by identified the type then the function name with param brackets () then the code inside the {} brackets the next complex code aims to achive this simple task 

In [8]:
%%writefile test.cpp
void Binstar::synchronise_dt_star(){
    //TODO Add other case as isempty, is naked.
    if ( (star[0]->getp(Worldtime::ID)!=star[1]->getp(Worldtime::ID))||(star[0]->getp(Worldtime::ID)!=getp(BWorldtime::ID)))
        svlog.critical("Stellar and/or Binary Worldtimes are not sync S0_T0=" +std::to_string(star[0]->getp_0(Worldtime::ID)) +
        " S0_T=" +std::to_string(star[0]->getp(Worldtime::ID)) + " S1_T0="+std::to_string(star[1]->getp_0(Worldtime::ID)) +
                               " S1_T="+std::to_string(star[1]->getp(Worldtime::ID)) +
                               " B_T0=" + std::to_string(getp_0(BWorldtime::ID)) + " B_T=" +std::to_string(getp(BWorldtime::ID)),__FILE__,__LINE__);

    //Now the remnant tmax is handled directly in Timestep
    //if (star[0]->isremnant) star[0]->sync_with(tf-star[0]->getp(Worldtime::ID));
    //if (star[1]->isremnant) star[1]->sync_with(tf-star[1]->getp(Worldtime::ID));
    //We can leave this check because if broken the Binary properties are not evolved.
    if (broken) property[BTimestep::ID]->resynch(tf-star[0]->getp(Worldtime::ID));
    //property[BTimestep::ID]->resynch(1e30, false);  //TODO I have to evolve Btimestep
    double mindt;
    mindt = std::min({star[0]->getp(Timestep::ID), star[1]->getp(Timestep::ID),property[BTimestep::ID]->get()});
    star[0]->sync_with(mindt);
    star[1]->sync_with(mindt);
    property[BTimestep::ID]->resynch(mindt, false);
}


Overwriting test.cpp


In [9]:
class_list= [key for key in all_classes]#dictionary to simple list
typedef_list = [key for key in all_typedefs]

def is_code_defined(line): ##test if the next readable part is a {
    code_started, empty_line = False, False
    i=0
    for ch in line:# we need 
        if ch != ' ' and ord(ch)!=10:#skip emptys
            empty_line=False
            if ch=='{':
                code_started=True
                #print('code started :)')
                break
            else: 
                #print("function not defined")
                break
        i+=1

    if len(line)==i:
        empty_line=True
    return code_started, empty_line

def reset_find():
    params_closed_here, params_open,code_started, find_code_start,code_passed = False, False, False, False, False
    type_count, l, r, l_f, r_f =0,0,0,0,0
    reset=False
    params=''
    return  params_closed_here, params_open,code_started, find_code_start,code_passed,reset, type_count, l, r, l_f, r_f, params

def code_start_test(line):
    l,r=0,0
    reset=False
    code_passed=False
    code_finished=False
    code_started, empty_line = is_code_defined(line)
    if code_started and not empty_line:#code started and empty line only used here 
        #print('code passed')
        code_passed=True
        l,r, code_finished = bracket_counter(line)
    if not code_started and not empty_line:
        reset=True
    return code_passed, l,r, code_finished, reset

def type_from_file(word):
    sub_words=word.split('::')
    origin=sub_words[:-1]
    output=sub_words[-1]
    return '::'.join(origin), output


def find_functions(file_list):
    #This needs more work as not at all robust or comprehansive
    #just prelimary draft
    
    if type(file_list) is not list: file_list = [file_list] 
    
    #make a list of the types
    c_types=['short int', 'unsigned short int', 'unsigned int','int', 
             'long int', 'unsigned long int', 'long long int',
             'unsigned long long int', 'signed char', 'unsigned char', 'float',
             'double', 'long double', 'void']
    std_types=['size_t', 'wchar_t','div_t', 'ldiv_t' '::']
    sevn_types=class_list+c_types+std_types+typedef_list
    function_properties={}
    
    for path in file_list:
        #Define all the function parameters
        params_closed_here, params_open,code_started, find_code_start, code_passed = False, False, False, False, False
        type_count, l, r, l_f, r_f =0,0,0,0,0
        params=''
        i=1
        in_comment=False
        f = open(path , "r")
        for line in f:
            pos, comment, in_comment=check_comment(line, in_comment) #only process uncommented lines
            line=line[pos[0]:pos[1]]
            line=line.replace('){',') {')
            #print('line:' +str(i)+line)
            if not in_comment:
                if not find_code_start:
                    words=line.split()#split on spaces 
                    for word in words:#inside line 
                        #first find check if starts with types  --------
                        if not params_open:
                            type_word=word#for use with type test
                            if '::' in word and type_count==0:#detected types imported from other files 
                                type_from, type_word= type_from_file(word)

    ## 1 find types 
                            if sevn_types.count(type_word)>0:#1 or more types found 
                                #print(type_word)
                                #print('found type', word)
                                type_count +=1
                                type_out=type_word
                                #print(type_count)
                                #if the next word is not a type and does not have bracket RESET 
                                #also assumes start on same line
                                if words.index(word) + 1< len(words):
                                    if '(' not in words[words.index(word) + 1] and sevn_types.count(word)==0 or params_closed_here:
                                        print('reset on', word)
                                        params_closed_here, params_open,code_started, find_code_start, \
                                        code_passed,reset, type_count, l, r, l_f, r_f, params =  reset_find()
    ## 2 params ( )        
                            #Second look for the param input () --------
                            if type_count>0 and sevn_types.count(type_word)==0:#first part after types look for ()
                                if '(' in word: 
                                    f_start = i
                                    #print('function detected i =', i)
                                    l_f, r_f, params_closed_here = bracket_counter(word, l_f, r_f, params_closed_here, brackets='()')
                                    #print('params start line ', l_f,r_f)
                                    fun_name=word[:word.find('(')]
                                    if not params_closed_here:
                                        params_open=True
                                        params+=word[word.find('(')+1:]#1st param word  and trim function name of

                                    else: #param bracket () closed on start word
                                        params_open=False
                                        find_code_start=True
                                        param_end = params.rfind(')')
                                        params=params[:param_end]
                                        line_ = line[line.rfind(')')+1:]
                                        #params ended same word look for code
                                        code_passed, l,r, code_finished, reset = code_start_test(line_)
                                        if code_passed:
                                            #print("passed straight away")
                                            #print(find_code_start)
                                            break #escape word by word part
                                        if reset:
                                            #print("reset g")
                                            params_closed_here, params_open,code_started, find_code_start, \
                                            code_passed,reset, type_count, l, r, l_f, r_f, params =  reset_find()

                                else: #not type and ( not found -> reset 
                                    params_closed_here, params_open,code_started, find_code_start, \
                                    code_passed,reset, type_count, l, r, l_f, r_f, params =  reset_find()

                        #if params_open 
                        else:# Track () closure and record subsequent parameters 
                            #print(i, 'param detection')
                            params+=word+' ' #record each word 
                            l_f, r_f, params_closed_here = bracket_counter(word, l_f, r_f, params_closed_here, brackets='()')
                            if params_closed_here:
                                #print('B params closed same line subsequent word after type')
                                param_end = params.rfind(')')
                                params = params[:param_end]
                                params = params.split(',')
                                params_open=False
                                l_f, r_f, type_count = 0,0,0
                                #print('params', params)
                                #Now check code start (same line subsequent word)
                                find_code_start=True
                                line_ = line[line.rfind(')')+1:]
                                code_passed, l,r, code_finished, reset = code_start_test(line_)
                                if reset:
                                    params_closed_here, params_open,code_started, find_code_start, \
                                    code_passed,reset, type_count, l, r, l_f, r_f, params =  reset_find()

                #line by line 
                #if find code start
                #to get to here needs a new line 
                else:# inside {} the code for the function has begun
                    #trim line
                    #print('Line by line')
                    if not code_passed:
                        #print('Final Block')    
                        code_passed, l,r, code_finished, reset = code_start_test(line)
                        if reset:
                            params_closed_here, params_open,code_started, find_code_start, \
                            code_passed,reset, type_count, l, r, l_f, r_f, params =  reset_find()

                    #code_passed the test
                    else:
                        l,r, code_finished = bracket_counter(line, l,r)
                        if code_finished:
                            function_properties[fun_name]= {'type:':type_out, 'params': params, 'file': path, 'lines': (f_start,i)}
                            #reset here 
                            params_closed_here, params_open,code_started, find_code_start, \
                            code_passed,reset, type_count, l, r, l_f, r_f, params =  reset_find()
            i+=1
        f.close() 
    return function_properties

In [10]:
test_path='/home/jake/SEVN/sevndevel/src/binstar/binstar.cpp'
all_f=find_functions(all_cpp)
save_dict(all_f, 'all_functions.json')

find_functions(test_path)

reset on evolve_utility::EvolveFunctor
reset on Lookup::FILEMAP
reset on Lookup::FILEMAP
reset on Lookup::OUTPUTMAP
reset on Lookup::INPUTMAPBIN_NPARAM
reset on Lookup::INPUTMAPBIN
reset on Lookup::WINDSMAP
reset on Lookup::RLMAP
reset on Lookup::TIDESMAP
reset on Lookup::GWMAP
reset on Lookup::MIXMAP
reset on Lookup::SNKMAP
reset on Lookup::CEMAP
reset on std::size_t


{'Binstar::evolve_binary': {'type:': 'bse_evolution',
  'params': '',
  'file': '/home/jake/SEVN/sevndevel/src/binstar/binstar.cpp',
  'lines': (19, 224)},
 'Binstar::synchronise_dt_star': {'type:': 'void',
  'params': '',
  'file': '/home/jake/SEVN/sevndevel/src/binstar/binstar.cpp',
  'lines': (226, 248)},
 'Binstar::check_and_sync_sse': {'type:': 'void',
  'params': '',
  'file': '/home/jake/SEVN/sevndevel/src/binstar/binstar.cpp',
  'lines': (250, 280)},
 'Binstar::check_and_sync_bse': {'type:': 'void',
  'params': '',
  'file': '/home/jake/SEVN/sevndevel/src/binstar/binstar.cpp',
  'lines': (282, 359)},
 'Binstar::check_and_set_bavera_xspin': {'type:': 'void',
  'params': '',
  'file': '/home/jake/SEVN/sevndevel/src/binstar/binstar.cpp',
  'lines': (361, 378)},
 'Binstar::update_derived_properties_star': {'type:': 'void',
  'params': ['Star*s'],
  'file': '/home/jake/SEVN/sevndevel/src/binstar/binstar.cpp',
  'lines': (436, 444)},
 'Binstar::check_accretion_on_compact': {'type:': 